In [1]:
import os
import pandas as pd
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
# Chuyển dữ liệu landmark thành onehot vector, ví dụ: [1, 2, 3] -> [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
from tensorflow.keras.utils import to_categorical
# Chia dữ liệu train thành train và valid để chạy 
from sklearn.model_selection import train_test_split
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

**Class config bao gồm:**
1. IMAGE_SIZE: Kích thước input 
2. LANDMARK_IDS: Số landmark (class) được chọn (theo yêu cầu là 30-40% của 81313 => 25000
3. PATIENCE: Số lần chờ với early stopping
4. NUM_EPOCHS: Số epoch dùng để huấn luyện
5. ER_VERBOSE: Verbose cho early stopping
6. BATCH_SIZE: Lô huấn luyện
7. EST_DATA: Số lượng ảnh test cần test (tổng là ~10345 ảnh)
8. TRAIN_ROOT: Đường dẫn tới thư mục train
9. TEST_ROOT: Đường dẫn tới thư mục test

In [2]:
class CFG:
    IMAGE_SIZE=112 #112 448 
    LANDMARK_IDS=50
    PATIENCE=5
    NUM_EPOCHS=30
    ER_VERBOSE=1
    BATCH_SIZE=32
    TEST_DATA=4000
    TRAIN_ROOT='../input/landmark-recognition-2020/train'
    TEST_ROOT='../input/landmark-recognition-2020/test'

In [3]:
# Đọc dữ liệu bằng pandas
train_df = pd.read_csv("../input/landmark-recognition-2020/train.csv")
test_df = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
train_df.head(5)

In [4]:
len(test_df)

In [5]:
# Hiển thị kích thước dữ liệu:
# Hàng: Số ảnh
# Cột ids + landmark ids
print('Train data shape:', train_df.shape)
print('Test data shape:', test_df.shape)

In [6]:
# Hiển thị số class
print("Len unique train label:", len(train_df['landmark_id'].unique()))

In [8]:
# Hiển thị một vài landmark id trong tập huấn luyện mình lấy ra
landmark_unique = train_df['landmark_id'].unique()
landmark_unique[0: CFG.LANDMARK_IDS]

**Hàm này sẽ phân tích số ảnh trong mỗi class (số ảnh chứa landmark cụ thể)**

In [9]:
def samples_distribution(value_counts: pd.Series):
    mean_images = round(value_counts.mean(), 0)
    median_images = round(value_counts.median(), 0)
    print(f'Total number of classes: {len(value_counts)}')
    print(f'{value_counts.min()} - {value_counts.max()} samples per class')
    print(f'Mean value: {mean_images} samples\n')
    images_per_class.hist(bins=20, log=True)
    plt.vlines(mean_images, ymin=0, ymax=80_000, colors='red', label='Mean number')
    plt.vlines(median_images, ymin=0, ymax=80_000, colors='green', label='Median number')
    plt.title('Train samples per class')
    plt.xlabel('Number of images')
    plt.ylabel('Frequency')
    plt.show()

In [10]:
images_per_class = train_df['landmark_id'].value_counts()
n_classes = len(images_per_class)
samples_distribution(images_per_class)

**Lấy ra id của những ảnh mình huấn luyện**

In [11]:
from tqdm import tqdm
train_image_ids = []
test_image_ids = []
labels = []
temp_labels = []
i = 0
for id_ in tqdm(landmark_unique[0:CFG.LANDMARK_IDS]):
    for id in train_df['id'][train_df['landmark_id'] == id_]:
        train_image_ids.append(id)
        labels.append(id_)
        temp_labels.append(i)
    i = i+1
len(train_image_ids)


**Lấy ra id của tất cả ảnh test**

In [12]:
test_image_ids = test_df['id']

In [13]:
train_image_ids[0]

In [14]:
test_image_ids[0]

**Hàm get_image_path sẽ lấy ra kết quả:**
1. List đường dẫn tới các ảnh (train/ test)
2. Ảnh (train/ test)

In [15]:
from tqdm import tqdm

def get_image_path(root, ids):
    image_paths = []
    images = []
    for i in tqdm(range(len(ids))):
        # lấy image path từ thư mục root
        first_dir = os.path.join(root,ids[i][0])
        second_dir = os.path.join(first_dir,ids[i][1])
        third_dir = os.path.join(second_dir,ids[i][2])
        final_path = os.path.join(third_dir,ids[i]+'.jpg')
        #đọc ảnh = opencv
        img = cv2.imread(final_path)[:,:,::-1]
        #resize ảnh về CFG_IMAGE_SIZE e.g = 224
        images.append(cv2.resize(img, (CFG.IMAGE_SIZE, CFG.IMAGE_SIZE)))
        image_paths.append(final_path)
    r = {
        'image_paths': image_paths,
        'images': images
    }
    return r

In [16]:
r_train = get_image_path(CFG.TRAIN_ROOT, train_image_ids)
r_test = get_image_path(CFG.TEST_ROOT, test_image_ids)

In [17]:
train_image_paths, train_images = r_train['image_paths'], r_train['images']
test_image_paths, test_images = r_test['image_paths'], r_test['images']

**Plot trực quan một số ảnh trong tập train**

In [18]:
fig = plt.gcf()
fig.set_size_inches(16, 16)
next_pix_ = train_image_paths
for i, img_path in enumerate(next_pix_[0:16]):
    sp = plt.subplot(5, 4, i + 1)
    sp.axis('Off')
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

**Plot trực quan một số ảnh trong tập test**

In [19]:
fig = plt.gcf()
fig.set_size_inches(16, 16)
next_pix_ = test_image_paths
for i, img_path in enumerate(next_pix_[0:16]):
    sp = plt.subplot(5, 4, i + 1)
    sp.axis('Off')
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

In [20]:
#tạo image và labels
#https://www.w3schools.com/python/ref_func_zip.asp
shuf = list(zip(train_images,temp_labels))
random.shuffle(shuf)
train_data, labels_data = zip(*shuf)
print('Images: ', len(train_data))
print('Labels: ', len(labels_data))

In [21]:
#ma trận của 1 ảnh
# train_data[0]

In [22]:
#landmark_id hoặc class
labels_data[0]

In [23]:
# chuẩn hóa ảnh
X_train_data = np.array(train_data) / 255
# one hot
Y_data = to_categorical(labels_data, num_classes=50) 
# chia train test
X_train, X_val, Y_train, Y_val = train_test_split(X_train_data, Y_data, test_size=0.3, random_state=42)

In [24]:
# Chuẩn hóa ảnh test
X_test_data = test_images[0: CFG.TEST_DATA]
test_image_ids = test_image_ids[0: CFG.TEST_DATA]
X_test_data = np.array(X_test_data) / 255

**Sử dụng augment cho ảnh train**

In [25]:
#augment: https://github.com/albumentations-team/albumentations
#https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.5,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.3,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None
)

**TRAINING**

In [37]:
# Tạo mô hình huấn luyện dữ liệu
# lấy pretrained từ tập imagenet 1K https://image-net.org/download.php
# finetune lại mô hình (mình đã có các features từ tập ảnh imagenet)
model = tf.keras.applications.DenseNet201(input_shape=(CFG.IMAGE_SIZE,CFG.IMAGE_SIZE,3),
                                                      include_top=False,
                                                      weights='imagenet',
                                                      pooling='avg')
model.trainable = False
inputs = model.input
drop_layer = tf.keras.layers.Dropout(0.2)(model.output)
x_layer = tf.keras.layers.Dense(512, activation='relu')(drop_layer)
x_layer1 = tf.keras.layers.Dense(128, activation='relu')(x_layer)
drop_layer1 = tf.keras.layers.Dropout(0.20)(x_layer1)
outputs = tf.keras.layers.Dense(50, activation='softmax')(drop_layer1)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [27]:
# model.summary()

In [38]:
# Thuật toán tối ưu: Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# Loss CE https://tonydeep.github.io/tensorflow/2017/07/07/Cross-Entropy-Loss.html
# Hàm loss sử dụng: Cross Entropy Loss
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['acc'])
# Kỹ thuật dừng sớm tránh overfitting
# https://machinelearningcoban.com/2017/03/04/overfitting/
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=CFG.PATIENCE,
    verbose=CFG.ER_VERBOSE,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)
# Tiến hành fit data và huấn luyện mô hình
history = model.fit(datagen.flow(X_train,Y_train,batch_size=CFG.BATCH_SIZE),\
                    validation_data=(X_val,Y_val),\
                    epochs=CFG.NUM_EPOCHS,\
                    callbacks=[early_stopping])

**Visualize đồ thì accuracy của quá trình huấn luyện (training) và kiểm thử (validation)**

In [29]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Train accuracy')
plt.plot(epochs, val_acc, 'b', label='Valid accuracy')
plt.title('Training log')
plt.legend(loc=0)
plt.figure()


plt.show()

**Visualize đồ thì lỗi của quá trình huấn luyện (training) và kiểm thử (validation)**

In [30]:
plt.plot(epochs, loss, 'r', label='Train loss')
plt.plot(epochs, val_loss, 'b', label='Valid loss')
plt.title('Training log')
plt.legend(loc=0)
plt.figure()
plt.show()

**INFERENCE**

In [31]:
# Kiểm tra đầu vào tập test
X_test_data.shape

In [32]:
# Tiến hành predict dữ liệu
result = model.predict(X_test_data)
result = np.argmax(result, axis=1)

**Ghi kết quả dự đoán ra file csv**

In [33]:
result_csv = pd.DataFrame(columns=['id', 'landmark_id'])
result_csv['id'] = test_image_ids
result_csv['landmark_id'] = result
result_csv

**Thu gom bộ nhớ**

In [34]:
import gc
gc.collect()